# Intersight REST API Query Parameter Usage Examples

Intersite supports the use of a query language in the URI of a REST API request, as query parameters.  This Notebook includes examples of using the Intersight query language to **filter results returned to the REST client, on the server/Intersight side**.  This process can simplify the effort to filter results on the client side.

- Filter options include:
    - `eq`, `ne`, `gt`, `lt`, `ge`, `le`
        - `$filter=NumCpus ge 4`
    - `and`, `or`, `not`
        - `($filter=NumCpus ge 4 and NumCpus le 8)`
    - `in`
        - `($filter=Model in ('HX220C-M5SX', 'UCSC-C240-M5SN')`
    - `contains`
        - `($filter=contains(Model, B200))`
    - `startswith`
        - `($filter=startswith(Model, UCSC))`
    - `endswith`
        - `($filter=endswith(Model, M5))`
    - `tolower`/`toupper`
        - `($filter=contains(Model, toupper(b200)))`

Reference: [Intersight Query Syntax](https://intersight.com/apidocs/introduction/query/ "Intersight Query Syntax")

---

## Import the `intersight_helper` functions

### Intersight API Authentication Setup

**IMPORTANT**

Be sure to follow the instructions [**here**](https://wwt.github.io/dcauto-study-resources/sections/section_4/#hands-on-learning ) before you attempt to run the next and subsequent cells in this notebook.

In [1]:
# Setup exception handling for the Intersight authentication process
try:

    # Attempt to import the functions in the `intersight_`helper` module - this import attempts Intersight authentication.
    from intersight_helper import *
    from requests import HTTPError

# Handle missing keyId.txt and keySecret.txt files
except FileNotFoundError as e:
    print('Unable to locate authentication key and signature file combination.')
    print(f'{e!r}')
    print(f'{e.filename!r}')

### Test to determine if authentication and authorization are successful.

In [2]:
# Determine if the keyId.txt and keySecret.txt files successfully authenticate
try:
    results = intersight(
    method='GET',
    endpoint='/compute/Blades'
    )
except HTTPError as e:
    print('The Intersight authentication process failed.\n'
          'Please be sure to follow the "Intersight API Authentication Setup" directions above before continuing.')

# The Intersight API authentication and authorization process is successful if you see `200 OK` below


HTTP response: 200 OK
Objects returned: 18



---

## Create a helper function to display query results.

In [3]:
from typing import Dict, List


def display_results(
    results: Dict,
    fields: List = None
) -> None:
    """ Helper function to display query results.

        Args:
            results (Dict):
                Dictionary of results returned by the Intersight
                API call.

            fields (List, optional):
                Optional List of dictionary fields to display.
                'Dn' is automatically included.

        Returns:
            None.
    """

    # Alias results['Results']
    results = results.get('Results', list())

    # Check count of results
    if len(results) == 0:
        print('No results found.')
        return

    for index, result in enumerate(results):
        # Display current and total results count
        print(f'{index + 1} of {len(results)}')

        # Display the 'Dn' for each result
        print(f'DN: {result.get("Dn", "N/A")}')

        # Display any optional fields
        if fields is not None:
            for field in fields:
                print(f'\t{field}: {result.get(field, "N/A")}')
            print()

---

## Examples

### Filter for specific resources, using matching criteria

In [4]:
results = intersight(
  method='GET',
  endpoint='/compute/Blades',
  params='$filter=NumCpus ge 4'
)

# Display the results
display_results(
    results=results,
    fields=[
        'Model',
        'NumCpus'
    ]
)


HTTP response: 200 OK
Objects returned: 2

1 of 2
DN: sys/chassis-5/blade-3
	Model: UCSB-B480-M5
	NumCpus: 4

2 of 2
DN: sys/chassis-5/blade-3
	Model: UCSB-B480-M5
	NumCpus: 4



---

### Return/select only certain matching properties

In [5]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$select=Model,Dn,Serial'
)

# Display the results
display_results(
    results=results,
    fields=[
        'Model',
        'Serial'
    ]
)


HTTP response: 200 OK
Objects returned: 18

1 of 18
DN: sys/chassis-5/blade-1
	Model: UCSB-B200-M5
	Serial: SRV122

2 of 18
DN: sys/chassis-5/blade-3
	Model: UCSB-B480-M5
	Serial: SRV125

3 of 18
DN: sys/chassis-3/blade-1
	Model: UCSB-EX-M4-1
	Serial: SRV107

4 of 18
DN: sys/chassis-3/blade-3
	Model: UCSB-EX-M4-1
	Serial: SRV108

5 of 18
DN: sys/chassis-3/blade-7
	Model: UCSB-EX-M4-1
	Serial: SRV110

6 of 18
DN: sys/chassis-4/blade-1
	Model: UCSC-C3K-M4SRB
	Serial: SRV111

7 of 18
DN: sys/chassis-4/blade-2
	Model: UCSC-C3K-M4SRB
	Serial: SRV112

8 of 18
DN: sys/chassis-3/blade-5
	Model: UCSB-EX-M4-1
	Serial: SRV126

9 of 18
DN: sys/chassis-5/blade-2
	Model: UCSB-B200-M5
	Serial: SRV124

10 of 18
DN: sys/chassis-3/blade-1
	Model: UCSB-EX-M4-1
	Serial: SRV107

11 of 18
DN: sys/chassis-3/blade-5
	Model: UCSB-EX-M4-1
	Serial: SRV126

12 of 18
DN: sys/chassis-4/blade-2
	Model: UCSC-C3K-M4SRB
	Serial: SRV112

13 of 18
DN: sys/chassis-5/blade-3
	Model: UCSB-B480-M5
	Serial: SRV125

14 of 18


---

### Pagination, return the top N results only

In [6]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$top=3'
)

# Display the results
display_results(
    results=results,
    fields=[
        'Model',
        'Serial',
        'NumThreads'
    ]
)


HTTP response: 200 OK
Objects returned: 3

1 of 3
DN: sys/chassis-5/blade-1
	Model: UCSB-B200-M5
	Serial: SRV122
	NumThreads: 16

2 of 3
DN: sys/chassis-5/blade-3
	Model: UCSB-B480-M5
	Serial: SRV125
	NumThreads: 32

3 of 3
DN: sys/chassis-3/blade-1
	Model: UCSB-EX-M4-1
	Serial: SRV107
	NumThreads: 16



---

### Pagination, skip the top N results only

In [7]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$skip=3'
)

# Display the results
display_results(
    results=results,
    fields=[
        'Model',
        'Serial',
        'NumThreads'
    ]
)


HTTP response: 200 OK
Objects returned: 15

1 of 15
DN: sys/chassis-3/blade-3
	Model: UCSB-EX-M4-1
	Serial: SRV108
	NumThreads: 16

2 of 15
DN: sys/chassis-3/blade-7
	Model: UCSB-EX-M4-1
	Serial: SRV110
	NumThreads: 16

3 of 15
DN: sys/chassis-4/blade-1
	Model: UCSC-C3K-M4SRB
	Serial: SRV111
	NumThreads: 16

4 of 15
DN: sys/chassis-4/blade-2
	Model: UCSC-C3K-M4SRB
	Serial: SRV112
	NumThreads: 16

5 of 15
DN: sys/chassis-3/blade-5
	Model: UCSB-EX-M4-1
	Serial: SRV126
	NumThreads: 16

6 of 15
DN: sys/chassis-5/blade-2
	Model: UCSB-B200-M5
	Serial: SRV124
	NumThreads: 16

7 of 15
DN: sys/chassis-3/blade-1
	Model: UCSB-EX-M4-1
	Serial: SRV107
	NumThreads: 16

8 of 15
DN: sys/chassis-3/blade-5
	Model: UCSB-EX-M4-1
	Serial: SRV126
	NumThreads: 16

9 of 15
DN: sys/chassis-4/blade-2
	Model: UCSC-C3K-M4SRB
	Serial: SRV112
	NumThreads: 16

10 of 15
DN: sys/chassis-5/blade-3
	Model: UCSB-B480-M5
	Serial: SRV125
	NumThreads: 32

11 of 15
DN: sys/chassis-3/blade-3
	Model: UCSB-EX-M4-1
	Serial: SRV

---

### Pagination, skip the first N results and return only the top X remaining results

In [8]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$skip=3&$top=5'
)

# Display the results
display_results(
    results=results,
    fields=[
        'Model',
        'Serial',
        'NumThreads'
    ]
)


HTTP response: 200 OK
Objects returned: 5

1 of 5
DN: sys/chassis-3/blade-3
	Model: UCSB-EX-M4-1
	Serial: SRV108
	NumThreads: 16

2 of 5
DN: sys/chassis-3/blade-7
	Model: UCSB-EX-M4-1
	Serial: SRV110
	NumThreads: 16

3 of 5
DN: sys/chassis-4/blade-1
	Model: UCSC-C3K-M4SRB
	Serial: SRV111
	NumThreads: 16

4 of 5
DN: sys/chassis-4/blade-2
	Model: UCSC-C3K-M4SRB
	Serial: SRV112
	NumThreads: 16

5 of 5
DN: sys/chassis-3/blade-5
	Model: UCSB-EX-M4-1
	Serial: SRV126
	NumThreads: 16



---

### Return objects in a certain order (select only certain properties)

In [9]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$orderby=Serial&$select=Dn,Serial,Model'
)

# Display the results
display_results(
    results=results,
    fields=[
        'Model',
        'Serial'
    ]
)


HTTP response: 200 OK
Objects returned: 18

1 of 18
DN: sys/chassis-3/blade-1
	Model: UCSB-EX-M4-1
	Serial: SRV107

2 of 18
DN: sys/chassis-3/blade-1
	Model: UCSB-EX-M4-1
	Serial: SRV107

3 of 18
DN: sys/chassis-3/blade-3
	Model: UCSB-EX-M4-1
	Serial: SRV108

4 of 18
DN: sys/chassis-3/blade-3
	Model: UCSB-EX-M4-1
	Serial: SRV108

5 of 18
DN: sys/chassis-3/blade-7
	Model: UCSB-EX-M4-1
	Serial: SRV110

6 of 18
DN: sys/chassis-3/blade-7
	Model: UCSB-EX-M4-1
	Serial: SRV110

7 of 18
DN: sys/chassis-4/blade-1
	Model: UCSC-C3K-M4SRB
	Serial: SRV111

8 of 18
DN: sys/chassis-4/blade-1
	Model: UCSC-C3K-M4SRB
	Serial: SRV111

9 of 18
DN: sys/chassis-4/blade-2
	Model: UCSC-C3K-M4SRB
	Serial: SRV112

10 of 18
DN: sys/chassis-4/blade-2
	Model: UCSC-C3K-M4SRB
	Serial: SRV112

11 of 18
DN: sys/chassis-5/blade-1
	Model: UCSB-B200-M5
	Serial: SRV122

12 of 18
DN: sys/chassis-5/blade-1
	Model: UCSB-B200-M5
	Serial: SRV122

13 of 18
DN: sys/chassis-5/blade-2
	Model: UCSB-B200-M5
	Serial: SRV124

14 of 1

---

### Return only a count of the matching objects, no objects or their properties

In [10]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$count=true'
)

# Display the results
print(f'Total matching objects: {results.get("Count")}')


HTTP response: 200 OK
Objects returned: 1

Total matching objects: 18


---

### Return a count of matching objects with objects and their properties

In [11]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$inlinecount=allpages&$select=Dn,Model,Serial'
)

# Display the results
print(f'Total matching objects: {results.get("Count")}\n')

display_results(
    results=results,
    fields=[
        'Model',
        'Serial',
        'NumThreads'
    ]
)


HTTP response: 200 OK
Objects returned: 18

Total matching objects: 18

1 of 18
DN: sys/chassis-5/blade-1
	Model: UCSB-B200-M5
	Serial: SRV122
	NumThreads: N/A

2 of 18
DN: sys/chassis-5/blade-3
	Model: UCSB-B480-M5
	Serial: SRV125
	NumThreads: N/A

3 of 18
DN: sys/chassis-3/blade-1
	Model: UCSB-EX-M4-1
	Serial: SRV107
	NumThreads: N/A

4 of 18
DN: sys/chassis-3/blade-3
	Model: UCSB-EX-M4-1
	Serial: SRV108
	NumThreads: N/A

5 of 18
DN: sys/chassis-3/blade-7
	Model: UCSB-EX-M4-1
	Serial: SRV110
	NumThreads: N/A

6 of 18
DN: sys/chassis-4/blade-1
	Model: UCSC-C3K-M4SRB
	Serial: SRV111
	NumThreads: N/A

7 of 18
DN: sys/chassis-4/blade-2
	Model: UCSC-C3K-M4SRB
	Serial: SRV112
	NumThreads: N/A

8 of 18
DN: sys/chassis-3/blade-5
	Model: UCSB-EX-M4-1
	Serial: SRV126
	NumThreads: N/A

9 of 18
DN: sys/chassis-5/blade-2
	Model: UCSB-B200-M5
	Serial: SRV124
	NumThreads: N/A

10 of 18
DN: sys/chassis-3/blade-1
	Model: UCSB-EX-M4-1
	Serial: SRV107
	NumThreads: N/A

11 of 18
DN: sys/chassis-3/blade

---

### Group objects by properties and aggregate results by values

- Group all servers by Model and display the average number of CPUs per model.
- Supported aggregates include:
    - min, max, average, and sum

In [12]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$apply=groupby((Model), aggregate(NumCpus with average as AverageCpuCount))'
)

# Display the results
display_results(
    results=results,
    fields=[
        'Model',
        'AverageCpuCount'
    ]
)


HTTP response: 200 OK
Objects returned: 4

1 of 4
DN: N/A
	Model: UCSB-B200-M5
	AverageCpuCount: 2

2 of 4
DN: N/A
	Model: UCSB-B480-M5
	AverageCpuCount: 4

3 of 4
DN: N/A
	Model: UCSB-EX-M4-1
	AverageCpuCount: 2

4 of 4
DN: N/A
	Model: UCSC-C3K-M4SRB
	AverageCpuCount: 2



---

### Group objects by the total count of a value

- Group all servers by Model and display the total number of servers for each Model.

In [13]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$apply=groupby((Model), aggregate($count as TotalCountOfServerModel))'
)

display_results(
    results=results,
    fields=[
        'Model',
        'TotalCountOfServerModel'
    ]
)


HTTP response: 200 OK
Objects returned: 4

1 of 4
DN: N/A
	Model: UCSC-C3K-M4SRB
	TotalCountOfServerModel: 4

2 of 4
DN: N/A
	Model: UCSB-EX-M4-1
	TotalCountOfServerModel: 8

3 of 4
DN: N/A
	Model: UCSB-B480-M5
	TotalCountOfServerModel: 2

4 of 4
DN: N/A
	Model: UCSB-B200-M5
	TotalCountOfServerModel: 4



---

### Sort grouped objects with the `$orderby` parameter (using the `desc` keyword as an example)

- Group all servers by Model and display the total number of servers for each Model.

In [14]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$apply=groupby((Model), aggregate($count as TotalCountOfServerModel))&$orderby=TotalCountOfServerModel desc'
)

# Display the results
display_results(
    results=results,
    fields=[
        'Model',
        'TotalCountOfServerModel'
    ]
)


HTTP response: 200 OK
Objects returned: 4

1 of 4
DN: N/A
	Model: UCSB-EX-M4-1
	TotalCountOfServerModel: 8

2 of 4
DN: N/A
	Model: UCSB-B200-M5
	TotalCountOfServerModel: 4

3 of 4
DN: N/A
	Model: UCSC-C3K-M4SRB
	TotalCountOfServerModel: 4

4 of 4
DN: N/A
	Model: UCSB-B480-M5
	TotalCountOfServerModel: 2



---

### Include related resources with the queried resources


In [15]:
results = intersight(
	method='GET',
    endpoint='/compute/Blades',
    params='$expand=Parent'
)

# Display the results
result_sample = results.get('Results')[0]
print(f'Dn: {result_sample["Dn"]}')
print(f'Parent Dn: {result_sample["Parent"].get("Dn")}')


HTTP response: 200 OK
Objects returned: 18

Dn: sys/chassis-5/blade-1
Parent Dn: sys/chassis-5


---

### Search for resources

- Allows the use of `$top`, `$skip`, `$orderby`, `$filter`, `$select`, & `$count`

In [16]:
results = intersight(
	method='GET',
    endpoint='/search/SearchItems',
    # Single quotes required on the search string
    # Outer double quotes will work also, without escaping inner single quotes
    params='$filter=endswith(Dn,\'5\')'
)

# Display the results
display_results(
    results=results,
    fields=[
        'ObjectType',
        'EpDn',
        'Serial',
        'TotalMemory'
    ]
)


HTTP response: 200 OK
Objects returned: 50

1 of 50
DN: sys/chassis-3/blade-5
	ObjectType: compute.PhysicalSummary
	EpDn: N/A
	Serial: SRV126
	TotalMemory: 49152

2 of 50
DN: sys/rack-unit-5
	ObjectType: compute.PhysicalSummary
	EpDn: N/A
	Serial: RK58
	TotalMemory: 49152

3 of 50
DN: sys/rack-unit-5
	ObjectType: compute.RackUnit
	EpDn: N/A
	Serial: RK58
	TotalMemory: 49152

4 of 50
DN: sys/chassis-3/blade-5
	ObjectType: compute.Blade
	EpDn: N/A
	Serial: SRV126
	TotalMemory: 49152

5 of 50
DN: sys/chassis-3/blade-3/adaptor-1/ext-eth-5
	ObjectType: adapter.ExtEthInterface
	EpDn: sys/chassis-3/blade-3/fabric-B/pc-1287
	Serial: N/A
	TotalMemory: N/A

6 of 50
DN: sys/chassis-5/blade-1/adaptor-2/ext-eth-5
	ObjectType: adapter.ExtEthInterface
	EpDn: sys/chassis-5/blade-1/fabric-B/pc-1305
	Serial: N/A
	TotalMemory: N/A

7 of 50
DN: sys/chassis-3/blade-7/adaptor-2/ext-eth-5
	ObjectType: adapter.ExtEthInterface
	EpDn: sys/chassis-3/blade-7/fabric-B/pc-1284
	Serial: N/A
	TotalMemory: N/A

8 of 